In [1]:
import re, csv
import PyPDF2
#import openpyxl

In [8]:
# PDF's may be given with different numbers of budget (dollar) columns
N_BUDGET_COLS = 5
# 5 account number columns + 1 account description + N budget columns
MAX_BUDGET_ENTRY_COLS = 5 + 1 + N_BUDGET_COLS

In [2]:
pdf_path = 'data/FY23-Tentative-Budget-Expenditures-resave.pdf'

In [3]:
# read pdf file into list
raw_pages = []
with open(pdf_path, 'rb') as f:
    pdf = PyPDF2.PdfFileReader(f)
    for page in pdf.pages:
        raw_pages.append(page.extract_text())
    n_pages =  pdf.getNumPages()


Xref table not zero-indexed. ID numbers for objects will be corrected.


In [4]:
for page in pdf.pages[0:2]:
    page

In [5]:
n_pages

122

In [6]:
clean_lines = []
for i, page in enumerate(raw_pages):
    this_page = page.splitlines(keepends=True)
    for line in this_page:
        clean_line = line.replace('\xa0', ' ')
        clean_line = clean_line.strip()
        clean_lines.append(clean_line)

In [7]:
clean_lines[0:30]

['3frbud12.p 76-4 Urbana, IL 06/15/22 Page:1',
 '05.22.02.00.00 TENTATIVE BUDGET-EXPENSES  (Date: 6/2022)  9:52 AM',
 'Account Level 2022-23 2021-22 2021-22 2020-21 2020-21',
 'FDTLOC FUNC OBJ  SJ',
 'Description                           TENTATIVE   Original Budget     FYTD Activity   Original Budget     FYTD Activity',
 '10',
 'EDUCATION',
 '000000',
 'UNRESTRICTED',
 '10E000 0000 0000 00 000000',
 'SUMMARY ACCOUNT OFFSET',
 '',
 '',
 '-40.00',
 '',
 '4.34',
 '10E000 0000 1230 00 000000',
 '',
 '',
 '245.00',
 '',
 '754.50',
 '10E000 0000 1270 00 000000',
 '',
 '',
 '220.00',
 '',
 '',
 '10E000 0000 2160 00 000000',
 '']

In [9]:
# first line of new page = any chars + Urbana,IL + date + Page:# + end of line
new_page_pattern = '.*Urbana, IL \d\d/\d\d/\d\d Page:\d{1,3}$'
# budget line item = ##E### ####
budget_entry_pattern = '^\d{2}E\d{3} \d{4}'
# subtotal line item = ## ---
subtotal_line_pattern = '^\d{2} ---'

In [10]:
# save to these output variables
output_lines = []
funds = {}
accounts = {}

In [23]:
# iterate over all extracted lines of the pdf
i = 4500
i2 = 4700 #len(clean_lines)
while i < i2:
    current_line = clean_lines[i]
    if re.match(new_page_pattern, current_line):
        i += 1
        print(f'HEADER: {current_line}')
    # line 2 of header = start of string + 10 digits separated by decimals
    elif re.match('^\d\d\.\d\d\.\d\d\.\d\d\.\d\d', current_line):
        i += 1
        print(f'HEADER: {current_line}')
    # line 3 of header = begins with "Account Level"
    elif re.match('^Account Level', current_line):
        i += 1
        print(f'HEADER: {current_line}')
    # line 4 of header = begins with "FDTLOC"
    elif re.match('^FDTLOC', current_line):
        i += 1
        print(f'HEADER: {current_line}')
    # line 5 of header = begins with "Description"
    elif re.match('^Description', current_line):
        i += 1
        print(f'HEADER: {current_line}')
    # fund number = start of string + two digit number + end of string
    elif re.match('^\d{2}$', current_line):
        # this line is the fund number, next line is the fund name
        funds[current_line] = clean_lines[i+1]
        i += 2
        print(f'FUND number:{current_line} name:{funds[current_line]}')
    # account number = start of string + six digit number + end of string
    elif re.match('^\d{6}$', current_line):
        # edge case where the account does not have a corresponding description
        # and the next line is a budget entry
        if re.match(budget_entry_pattern, clean_lines[i+1]):
            accounts[current_line] = 'BLANK'
        # this line is the account number, next line is the account name            
        else:
            accounts[current_line] = clean_lines[i+1]
        i += 1
        print(f'ACCOUNT number:{current_line} name:{accounts[current_line]}')
    # budget line item = ##E### ####
    elif re.match(budget_entry_pattern, current_line):
        # account numbers
        budget_line = current_line.split()
        # if the next line is empty or numeric, the account description is blank in the PDF
        if clean_lines[i+1]:
            try:
                float(clean_lines[i+1].replace(',',''))
                budget_line.append('BLANK')
            except ValueError:
                pass
        else:
            budget_line.append('BLANK')
        print(clean_lines[i+1])
        # continue adding budget line entries until we reach either
        #    1) the next budget line item
        #    2) the next page
        #    3) the maximum number of columns (helps with edge cases with subtotals)
        while not (re.match(budget_entry_pattern, clean_lines[i+1]) or 
                   re.match(new_page_pattern, clean_lines[i+1])):
            i += 1
            budget_line.append(clean_lines[i].replace(',',''))
            if len(budget_line) == MAX_BUDGET_ENTRY_COLS:
                break
        i += 1
        print(i, 'ENTRY:', budget_line)
        output_lines.append(budget_line)
    # subtotal = ## --- ---- ----
    elif re.match(subtotal_line_pattern, current_line):
        subtotal_line = current_line.split()
        subtotal_line.append('SUBTOTAL')
        # continue adding budget line entries until we reach the maximum number of columns
        while True:
            i += 1
            subtotal_line.append(clean_lines[i].replace(',',''))
            if len(subtotal_line) == MAX_BUDGET_ENTRY_COLS:
                break

        i += 1
        print('SUBTOAL:', subtotal_line)
        output_lines.append(subtotal_line)
    else:
        i += 1
        print(f'SKIPPING: {current_line}')

HEADER: Account Level 2022-23 2021-22 2021-22 2020-21 2020-21
HEADER: FDTLOC FUNC OBJ  SJ
HEADER: Description                           TENTATIVE   Original Budget     FYTD Activity   Original Budget     FYTD Activity
FUND number:10 name:EDUCATION
ACCOUNT number:100000 name:DRIVERS EDUCATION
SKIPPING: DRIVERS EDUCATION
EMPL 2.2
4514 ENTRY: ['10E000', '1700', '2170', '28', '103370', 'EMPL 2.2', '750', '750.00', '150.31', '750.00', '146.92']
LIFE INSURANCE
4521 ENTRY: ['10E000', '1700', '2210', '28', '103370', 'LIFE INSURANCE', '30', '30.00', '3.45', '30.00', '3.68']
HEALTH INSURANCE
4528 ENTRY: ['10E000', '1700', '2220', '28', '103370', 'HEALTH INSURANCE', '6060', '6060.00', '2323.00', '4000.00', '2500.00']
DR TRAIN & OPERATING SUPL HS
4535 ENTRY: ['10E000', '1700', '4110', '28', '103370', 'DR TRAIN & OPERATING SUPL HS', '4000', '4000.00', '6688.72', '4000.00', '5859.68']
SKIPPING: 
SKIPPING: 
SKIPPING: ________________
SKIPPING: _________________
SKIPPING: _________________
SKIPPING: _